In [1]:
import pickle 
import numpy as np
from biopandas.pdb import PandasPdb
import os

In [26]:
directory = "COVID19Proteases"
no_of_query_pdbs = 152


def returnrelevantFFVectors(dire, number_of_query_pdbs):
    allq = pickle.load(open("%s/Collector/Data/AllQuery.df" %(dire), "rb"))
    print("Current Shape", allq.shape)
    allpocks = []
    allq_index = []
    # First find the indices of the pockets
    for i in range(number_of_query_pdbs):
        if i < 9:
            file = "000" +str(i+1)
        elif i <99:
            file = "00" +str(i+1)
        else:
            file = "0" +str(i+1)

        # Remove the Solvents etc. Can be expanded
        # Orignal Ligand-Protein Co-crystalstructure - remove solvents
        real = PandasPdb().read_pdb('%s/%s/%s.pdb' %(dire, file, file))
        real.df["HETATM"] = real.df["HETATM"].drop(index=np.where(real.df["HETATM"]["residue_name"] == "HOH")[0])
        real.df["HETATM"] = real.df["HETATM"].reset_index()
        real.df["HETATM"] = real.df["HETATM"].drop(index=np.where(real.df["HETATM"]["residue_name"] == "DMS")[0])
        real.df["HETATM"] = real.df["HETATM"].reset_index()
        real.df["HETATM"] = real.df["HETATM"].drop(index=np.where(real.df["HETATM"]["residue_name"] == "CL")[0])

        # Looks at the pockets
        pockets = [f.name for f in os.scandir('%s/%s/%s_out/pockets/' %(dire, file, file))]
        temp = []
        pocks = []
        # Divide by 2 because you have a .pdb and .pqr file for the same pocket - lazy coding
        for ps_n in range(len(pockets)//2):
            ppdb = PandasPdb().read_pdb('%s/%s/%s_out/pockets/pocket%s_atm.pdb' %(dire, file, file, ps_n+1))
            reds = ppdb.df["ATOM"].drop_duplicates(('residue_name', 'residue_number'))
            reds = reds[["residue_name", "residue_number"]]
            temp += [real.distance(xyz=ppdb.df["ATOM"].mean()[2:5].tolist() , records=('HETATM',)).min()]
        # find the top 3 pockets in proximity of 5.5 A if any - most of them return < 3 pockets
        if np.min(temp) < 5.5:
            pocks += [np.argmin(temp)+1]
        temp[np.argmin(temp)] = 100
        if np.min(temp) < 5.5:
            pocks += [np.argmin(temp)+1]
        temp[np.argmin(temp)] = 100
        if np.min(temp) < 5.5:
            pocks += [np.argmin(temp)+1]
        allpocks += [pocks]

        # if there are no close-by pockets to the native ligand continue to the next pdb
        
        if len(allpocks[i]) == 0:
            continue
        # all Feature Vectors 
        ptf_file = [x.split() for x in open("%s/%s/pocket_f99/%s.ptf" %(dire, file, file), "r").readlines()]
        ptf_file = [x[-1].split(":") for x in ptf_file]
        ptf_index = []
        
        # binding pocket coordinates pdb
        for j, k in enumerate(allpocks[i]):
            ppdb = PandasPdb().read_pdb('%s/%s/%s_out/pockets/pocket%s_atm.pdb' %(dire, file, file, k))
            reds = ppdb.df["ATOM"].drop_duplicates(('residue_name', 'residue_number'))
            reds = reds[["residue_name", "residue_number"]]

        # return Feature Vector indicies that have a pocket_1 match -> native ligand pocket
            for m, l in enumerate(ptf_file):
                temp = reds[l[0] == reds["residue_name"]]
                if not temp[int(l[2]) == temp["residue_number"]].empty:
                    ptf_index += [m]
        # extract Feature Vector index from original Query Df
        temp2 = allq[allq["ConformerID"] == file]
        allq_index += temp2.iloc[ptf_index].index.to_list()
        if i %10 ==0:
            print(i)
    allq_index.sort()
    allq = allq.iloc[allq_index]
    allq = allq.drop_duplicates(('Annotation', 'ConformerID'))
    print("Reduced Size", allq.shape)
    return allq

In [27]:
master_df = returnrelevantFFVectors(directory, no_of_query_pdbs)

Current Shape (21675, 488)
0


/Users/michaelsuarez/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Reduced Size (81, 488)


In [ ]:
pickle.dump(master_df, open("COVID19_all_envs.df", "wb"))

In [ ]:
# You can actually also reduce all the Environments down to one per residue - check how much the FFVectors differ first though
new = master_df.copy()
lol = new.drop_duplicates(('FxnCenter', 'Resid'))
pickle.dump(lol, open("COVID19_reduced_envs.df", "wb"))